# Integrantes

* Fabrizzio Mura
* Joaquin Tapia
* Matías Bugueño

## Librerias necesarias

In [1]:
from read_file import read_file
from TTP import GA, objective_function
from TTP.crossovers import SegmentSimple, PMXSimple
from TTP.mutations import ResettingScramble, ResettingSwamp
from TTP.Greedy import greedy_ttp
from TTP.utils import Item
from read_file import read_file
from copy import deepcopy
import numpy as np

## Funciones Necesarias

In [2]:
def get_GA_items(thief_items: list, items: list[Item]) -> np.ndarray:
    items_size = len(items)
    thief_items_size = len(thief_items)
    GA_items = np.zeros((items_size), dtype= int)

    for i in range(items_size):
        for j in range(thief_items_size):
            if items[i].get_house_id() == thief_items[j]:
                GA_items[i] = 1

    return GA_items

## Preparación de data

Con la ayuda de la función **read_file()** leemos los datos del problema y este nos retornará el nombre del mismo, la instancia de una clase ladrón la cuál será la encargada de manipular su mochila, los objetos robados y la disminución de velocidad por cada objeto robado, se recibirá una lista de intancias de la clase House que almacenan la posición de una casa y los objetos robados, también se recibe una lista de instancias de la clase Item que contiene el precio y el peso de cada item y por último se recibe el ratio utilizado para la función objetivo.

Por último se prepara un diccionario data que será utilizado para el algoritmo génetico que será presentado más adelante.

In [3]:
name, thief , cities , items, ratio = read_file("./data/a280-ttp/a280_n279_uncorr_02.ttp")

data = {'Houses': cities,
        'Items': items,
        'Thief': {
            'v_max': thief.get_v_max(),
            'v_min': thief.get_v_min(),
            'max_capacity': thief.get_max_weight()
        },
        'Ratio': ratio}

## Algoritmo Greedy

El algoritmo greedy o algoritmo voraz esta encargado de generar una solución rápida para ser de apoyo al algoritmo genético, este resuelve con programación dinámica el problema del TSP (Traveling Selsman Problem) y a través de un algoritmo greedy el problema knapsack buscando los objetos con mayor proporción $\displaystyle \frac{precio}{peso}$, y luego el resultado de este algoritmo es procesado por la función objetivo la cuál se calcula de la siguiente manera.

$\text{Función Objetivo} = \displaystyle \sum^{m}_{i = 1} p_i - ratio * \sum^{n - 1}_{i = 1} (t_{x_{i},x_{i + 1}}) + t_{x_{n}, x_{1}}$

donde:

$p_{i}: \text{precio del objeto actual}$

$x_{i}: \text{nodo actual}$

$\displaystyle t_{x_{i}, x_{j}} = \frac{d_{x_{i},x_{j}}}{\text{velocidad actual}}$

In [4]:
route, thief_items, total_time, profit, execution_time = greedy_ttp(deepcopy(thief), cities)
thief_items = [item.get_house_id() for item in thief_items]
thief_items = get_GA_items(thief_items, items)
sol = (route, thief_items)

tarjet = objective_function(data, sol)

print(f"Tiempo de ejecución: {round(execution_time, 3)}[s]")
print(f"Tiempo ladrón: {round(total_time, 3)}[s]")
print(f"Dinero: ${profit}")
print(f"Función objetivo: {round(tarjet, 3)}")
print(f'Ruta del ladron: {route}')

Tiempo de ejecución: 0.153[s]
Tiempo ladrón: 3948.711[s]
Dinero: $32847
Función objetivo: 2205.001
Ruta del ladron: [1, 280, 2, 3, 279, 278, 4, 277, 276, 275, 274, 273, 272, 271, 16, 17, 18, 19, 20, 21, 128, 127, 126, 125, 30, 31, 32, 29, 28, 27, 26, 22, 25, 23, 24, 14, 13, 12, 11, 10, 8, 7, 9, 6, 5, 260, 259, 258, 257, 254, 253, 208, 207, 210, 209, 252, 255, 256, 249, 248, 247, 244, 241, 240, 239, 238, 231, 232, 233, 234, 235, 236, 237, 246, 245, 250, 251, 230, 229, 228, 227, 226, 225, 224, 223, 222, 219, 218, 215, 214, 211, 212, 213, 216, 217, 220, 221, 203, 202, 200, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 15, 133, 132, 131, 130, 129, 154, 155, 153, 156, 152, 151, 177, 176, 181, 180, 179, 178, 150, 149, 148, 147, 146, 145, 199, 198, 197, 194, 195, 196, 201, 193, 192, 191, 190, 189, 188, 187, 185, 184, 183, 182, 161, 162, 163, 164, 165, 166, 167, 168, 169, 101, 100, 99, 98, 93, 94, 95, 96, 97, 92, 91, 90, 89, 109, 108, 

## Algoritmo Genético

Los algoritmos genéticos son algoritmos de optimización basados en los principios de la genética, en particular los de selección natural. Estos intentan de imitar la evolución de una espicie durante distintas generaciones tomando en cuenta los conceptos de cruzamiento, mutación y selección de padres para las siguientes épocas.

<img src='./images/GA_example.png' style='width: 600px'></img>

### Pseudo Código

<img src='./images/Pseudocodigo_GA.png' width= 600px></img>

### En nuestro caso

Nosotros hemos decidido implementar este algoritmo basandonos en el pseudocódigo anterior, utilizando programación orientada a objetos y el patrón de diseño Strategy para la creación nuevas estrategias de mutación y de cruzamiento y dejar un código mucho más ordenado.

En la selección de individuos hemos preferido hacer uso de una elección elitista, y las mutaciones como los cruzamientos serán explicados en la siguiente sección.

### Mutación

Se utiliza el archivo Mutation_Base.py como interface para poder utilizar el resto de mutaciones:

```py
from abc import abstractmethod
from abc import ABCMeta
from numpy import ndarray

class Mutation_Base(metaclass = ABCMeta):
    @abstractmethod
    def execute_knapsack(self, sol: ndarray, item_list: list, max_capacity: int) -> ndarray:
        pass

    @abstractmethod
    def execute_tsp(self, sol: ndarray) -> ndarray:
        pass

    @abstractmethod
    def execute(self, child: tuple, item_list: list, max_capacity: int) -> tuple:
        pass
```

Poseemos mutaciones tanto para los problemas TSP como para Knapsack individualmente.
1) Resetting (Knapsack)
2) Scramble (TSP)
3) Swamp (TSP)


1) Resetting ( just for Knapsack ):
	````py
	execute_knapsack(self, sol: list, item_list: list = None, max_capacity: int = 0) -> list:
		sol_size = len(sol)
		indexes = random.sample(range(sol_size), int(sol_size * 0.2))

		for index in indexes:
			if sol[index] == 1:
					sol[index] = 0
			else:
				item_weight = item_list[index].get_weight()
				total_weight = sum(item_list[j].get_weight() for j in range(len(sol)) if sol[j] == 1)
				if total_weight + item_weight <= max_capacity:
					sol[index] = 1
			return sol
	````

	Realizamos para una solución al problema knapsack [0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, ...] una mutación que genera una lista de tamaño int(sol_size * 0.2) que corresponde al 20% de los objetos del arreglo.

	Esta contiene los index de los objetos que serán luego modificados de manera binaria.

	<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRrfO6vZr3scj60HZTa8UKi8tPnVXximDOydg&s'></img>
	
	<pre style='font-size: smaller; color: yellow;'>

		if En caso de ser '1' el valor del objeto en el i-ésimo:
			-> El valor pasa a ser '0' (por lo que se elimina el ítem de la solución).
		else:
			-> Obtenemos el peso del item que intentaremos robar.
			-> Se calcula el peso de la mochila considerando el nuevo objeto ingresado.
			
			if El peso (considerando el objeto) no sobrepasa el de la mochila:
				-> el valor pasa a ser '1' (considerando como añadido el objeto).

	</pre>

2) Resetting Scramble( TSP-Mutation based on random reordering ):
	````py
	execute_tsp(self, sol: list) -> list:
		i, j = sorted(random.sample(range(len(sol)), 2))
		segmento = sol[i:j+1]
		random.shuffle(segmento)
		sol[i:j+1] = segmento
		
		return sol
	````

	Recibirá la solución al problema TSP correspondiente a una lista de n numeros ALL_DIFFERENT entre ellos, elementos que simulan la ruta seguida por el Thief.
	Se seleccionará en la primera linea dos valores un indice de inicio 'i' y un indice de termino 'j' para realizar la mutacion dentro de estos dos indices.

	<img src='https://www.researchgate.net/publication/329265512/figure/fig11/AS:941473017118758@1601476067356/Scramble-mutation-for-scheduling-the-jobs.png'></img>

	<pre style='font-size: smaller; color: yellow;'>
	-> Se generan dos indices aleatorios ALL_DIFFERENT. 
	-> Se genera un segmento de tamaño j+1-i o lo qe es igual, un segmento comprendido entre estos dos indices.
	-> Se hace un shuffle en los elementos del segmento para reordenarlos de manera aleatoria.
	-> Se añade finalmente el segmento a la solucion inicial.
	Se retorna la solución modificada
	</pre>

3) Resetting Swamp( TSP Mutation based on two-index swap):
	````py
	execute_tsp(self, sol: ndarray) -> ndarray:
		i, j = random.sample(range(len(sol)), 2)
		sol[i], sol[j] = sol[j], sol[i]

		return sol
	````
	<img src ='https://media.discordapp.net/attachments/1106395361867731026/1249602869045493761/image.png?ex=6667e6dc&is=6666955c&hm=5ca503e85f123b2b47d32ac909811fcfc5ce5c2ac26cb44fb26a180a52f66d26&=&format=webp&quality=lossless'></img>

	Aunque de manera similar a Scramble, se seleccionan dos indices par realizar un cambio, pero solamente entre estos dos elementos, cambiando solo dos 'ciudades' de la ruta.
	<pre style='font-size: smaller; color: yellow;'>
	-> Se generan dos elementos ALL_DIFFERENT aleatorios dentro del rango de ciudades posibles.
	-> Se intercambian los valores de estos dos elementos.
	Se retorna la solución.
	</pre>

<h3> Crossover </h3>

<h4> PMX (Partially Mapped Crossover) </h4>

El PMX se basa en la idea de intercambiar segmentos entre dos soluciones parentales (o cromosomas), garantizando que las ciudades en la ruta del TSP no se repitan y que cada ciudad sea visitada exactamente una vez.

El proceso para realizar este crossover consiste los siguientes pasos:

<img src= 'https://www.researchgate.net/publication/354532445/figure/fig4/AS:1070818071150594@1632314331440/Partially-mapped-crossover-PMX-operator.png'></img>

### Cruce de Segmentos Simple

Al igual que PMX también se seleccionan dos puntos de cruce, pero en este caso, los segmentos se intercambian directamente sin mapear los genes restantes.

Esto puede resultar en hijos con genes duplicados o faltantes, lo que no es válido en problemas de permutación como el TSP.

<img src= 'https://miro.medium.com/v2/resize:fit:1400/1*YhmzBBCyAG3rtEBbI0gz4w.jpeg'></img>

### Testing

Primero preparamos un diccionario del problema a resolver para que el algoritmo genético sepa los parametros a utilizar.

In [5]:
if 1 in route: route.remove(1)

problem_dict = {
    'obj_func': objective_function,
    'data': data,
    'mut_class': ResettingSwamp(),
    'cross_class': PMXSimple(),
    'init_pop': (deepcopy(route), deepcopy(thief_items))
}

In [6]:
optimizer = GA(epochs= 60, pop_size= 60, pc= 0.2, pm= 0.85, pb= 0.6)

In [7]:
best = optimizer.solve(problem_dict)

Epoch 1: Global best: 2538.207 | Current best: 2538.207 | Execution_time: 28.397[s]
Epoch 2: Global best: 2538.207 | Current best: 2538.207 | Execution_time: 17.75[s]
Epoch 3: Global best: 2876.432 | Current best: 2876.432 | Execution_time: 17.097[s]
Epoch 4: Global best: 3096.972 | Current best: 3096.972 | Execution_time: 17.458[s]
Epoch 5: Global best: 3096.972 | Current best: 2925.733 | Execution_time: 16.986[s]
Epoch 6: Global best: 3344.923 | Current best: 3344.923 | Execution_time: 17.005[s]
Epoch 7: Global best: 3756.165 | Current best: 3756.165 | Execution_time: 16.63[s]
Epoch 8: Global best: 3756.165 | Current best: 3756.165 | Execution_time: 17.524[s]
Epoch 9: Global best: 3756.165 | Current best: 3756.165 | Execution_time: 17.078[s]
Epoch 10: Global best: 3756.165 | Current best: 3756.165 | Execution_time: 16.821[s]
Epoch 11: Global best: 3756.165 | Current best: 3756.165 | Execution_time: 16.91[s]
Epoch 12: Global best: 3756.165 | Current best: 3756.165 | Execution_time: 16

Para mejorar las soluciones, se le agregó la función para la selección de población para las nuevas épocas, juntando los cromosomas de la generación anterior con la nueva, luego ordenando las soluciones con ayuda de la función objetivo y quedandonos con las mejores n cromosomas, además se aumentó la cantidad de items randomizados a cambiar en la mutación.

In [7]:
best = optimizer.solve(problem_dict)

Epoch 1: Global best: 2621.304 | Current best: 2621.304 | Execution_time: 45.06[s]
Epoch 2: Global best: 2621.304 | Current best: 2621.304 | Execution_time: 36.952[s]
Epoch 3: Global best: 2695.688 | Current best: 2695.688 | Execution_time: 34.969[s]
Epoch 4: Global best: 2695.688 | Current best: 2695.688 | Execution_time: 35.53[s]
Epoch 5: Global best: 3046.699 | Current best: 3046.699 | Execution_time: 31.242[s]
Epoch 6: Global best: 3046.699 | Current best: 3046.699 | Execution_time: 31.336[s]
Epoch 7: Global best: 3283.758 | Current best: 3283.758 | Execution_time: 30.327[s]
Epoch 8: Global best: 3397.255 | Current best: 3397.255 | Execution_time: 29.481[s]
Epoch 9: Global best: 3450.316 | Current best: 3450.316 | Execution_time: 29.421[s]
Epoch 10: Global best: 3972.727 | Current best: 3972.727 | Execution_time: 28.826[s]
Epoch 11: Global best: 3972.727 | Current best: 3972.727 | Execution_time: 29.65[s]
Epoch 12: Global best: 3972.727 | Current best: 3972.727 | Execution_time: 29

In [8]:
best = optimizer.solve(problem_dict)